In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from config.config import *
from utils.load_data_for_bas import *
from utils.utils import *
import torch
from torch.utils.data import DataLoader, SequentialSampler
from tqdm import tqdm
import pandas as pd
import random

from transformers import BertForSequenceClassification
from model.bertforsequence import *
from transformers import BertConfig, BertTokenizer


Devices: cuda, 1


I1122 01:54:02.752389 139969067321152 file_utils.py:39] PyTorch version 1.1.0 available.
I1122 01:54:02.972146 139969067321152 modeling_xlnet.py:194] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [3]:
tokenizer = BertTokenizer.from_pretrained(MODEL_PATH, do_lower_case=DO_LOWER_CASE)

I1122 01:54:05.397540 139969067321152 tokenization_utils.py:306] Model name 'model/bert_multilingual_cased_vn_finetuned' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc, bert-base-german-dbmdz-cased, bert-base-german-dbmdz-uncased). Assuming 'model/bert_multilingual_cased_vn_finetuned' is a path or url to a directory containing tokenizer files.
I1122 01:54:05.398846 139969067321152 tokenization_utils.py:335] Didn't find file model/bert_multilingual_cased_vn_finetuned/added_tokens.json. We won't load it.
I1122 01:54:05.399562 139969067321152 tokenization_utils.py:335] Didn't find file model/bert_multilingual_

In [4]:
import json
data = json.load(open("data/test.json",encoding='utf-8'))

In [12]:
examples = []
output_ids = []

for testcase in data:
    
    test_id = testcase["__id__"]
    text_a = testcase["question"]
    
    for paragraph in testcase["paragraphs"]:
        answer_id = paragraph["id"]
        text_b = testcase["title"] + " . " + paragraph["text"]
        output_ids.append({"test_id": test_id, "answer_id": answer_id, "question": text_a, "answer": text_b})
        
        example = InputExample(guid="", text_a=remove_nonlatin(text_a), text_b=remove_nonlatin(text_b), label=0)
        examples.append(example)
        
        t = text_b.split(" . ")
        t = [x.strip(".") for x in t]
        
#         if len(text_b) == 3:
#             t = [t[0], t[2], t[1]]
#             text_b = " . ".join(t)
#             output_ids.append({"test_id": test_id, "answer_id": answer_id, "question": text_a, "answer": text_b})
#             example = InputExample(guid="", text_a=remove_nonlatin(text_a), text_b=remove_nonlatin(text_b), label=0)
#             examples.append(example)
#         elif len(t) > 3:
#             random.shuffle(t)
#             text_b = " . ".join(t)
#             output_ids.append({"test_id": test_id, "answer_id": answer_id, "question": text_a, "answer": text_b})
#             example = InputExample(guid="", text_a=remove_nonlatin(text_a), text_b=remove_nonlatin(text_b), label=0)
#             examples.append(example)

#             random.shuffle(t)
#             text_b = " . ".join(t)
#             output_ids.append({"test_id": test_id, "answer_id": answer_id, "question": text_a, "answer": text_b})
#             example = InputExample(guid="", text_a=remove_nonlatin(text_a), text_b=remove_nonlatin(text_b), label=0)
#             examples.append(example)
            
            
        if len(t) == 3:
            t = [t[0], t[2], t[1]]
            output_ids.append({"test_id": test_id, "answer_id": answer_id, "question": text_a, "answer": " . ".join(t) + " ."})
            example = InputExample(guid="", text_a=remove_nonlatin(text_a), text_b=remove_nonlatin(" . ".join(t) + " ."), label=0)
            examples.append(example)
        elif len(t) > 3:
            current_text = [" . ".join(t) + " ."]
            print (current_text[0] + "\n")

            i = 0
            while (True):

                if i == 5:
                    break
                temp_1 = t[1:]
                random.shuffle(temp_1)
                temp_2 = [t[0]]
                temp_2.extend(temp_1)
                temp_text = " . ".join(temp_2) + " ."

                if temp_text not in current_text:
                    output_ids.append({"test_id": test_id, "answer_id": answer_id, "question": text_a, "answer": temp_text})
                    example = InputExample(guid="", text_a=remove_nonlatin(text_a), text_b=remove_nonlatin(temp_text), label=0)
                    examples.append(example)
                    current_text.append(temp_text)
                    i += 1
                    print (temp_text + "\n")
            print ("----")

The Landmark 81 . Toà tháp cao 81 tầng , hiện tại là toà nhà cao nhất Việt Nam và là toà nhà cao nhất Đông Nam Á từ tháng 3 năm 2018 . Dự án được xây dựng ở Tân Cảng , quận Bình Thạnh , ven sông Sài Gòn . Dự án được khởi công ngày 26/07/2014  .

The Landmark 81 . Toà tháp cao 81 tầng , hiện tại là toà nhà cao nhất Việt Nam và là toà nhà cao nhất Đông Nam Á từ tháng 3 năm 2018 . Dự án được khởi công ngày 26/07/2014  . Dự án được xây dựng ở Tân Cảng , quận Bình Thạnh , ven sông Sài Gòn .

The Landmark 81 . Dự án được xây dựng ở Tân Cảng , quận Bình Thạnh , ven sông Sài Gòn . Toà tháp cao 81 tầng , hiện tại là toà nhà cao nhất Việt Nam và là toà nhà cao nhất Đông Nam Á từ tháng 3 năm 2018 . Dự án được khởi công ngày 26/07/2014  .

The Landmark 81 . Dự án được khởi công ngày 26/07/2014  . Dự án được xây dựng ở Tân Cảng , quận Bình Thạnh , ven sông Sài Gòn . Toà tháp cao 81 tầng , hiện tại là toà nhà cao nhất Việt Nam và là toà nhà cao nhất Đông Nam Á từ tháng 3 năm 2018 .

The Landmark 81 

Madrid . Madrid ( ) là thủ đô và là thành phố lớn nhất của Tây Ban Nha . Dân số thành phố vào khoảng 3,4   triệu người , toàn bộ dân số của vùng đô thị Madrid ước tính khoảng 6,271 triệu người . Madrid là thành phố lớn thứ ba trong Liên minh châu Âu , sau Luân Đôn và Berlin  .

Madrid . Dân số thành phố vào khoảng 3,4   triệu người , toàn bộ dân số của vùng đô thị Madrid ước tính khoảng 6,271 triệu người . Madrid ( ) là thủ đô và là thành phố lớn nhất của Tây Ban Nha . Madrid là thành phố lớn thứ ba trong Liên minh châu Âu , sau Luân Đôn và Berlin  .

Madrid . Madrid là thành phố lớn thứ ba trong Liên minh châu Âu , sau Luân Đôn và Berlin  . Dân số thành phố vào khoảng 3,4   triệu người , toàn bộ dân số của vùng đô thị Madrid ước tính khoảng 6,271 triệu người . Madrid ( ) là thủ đô và là thành phố lớn nhất của Tây Ban Nha .

Madrid . Madrid ( ) là thủ đô và là thành phố lớn nhất của Tây Ban Nha . Madrid là thành phố lớn thứ ba trong Liên minh châu Âu , sau Luân Đôn và Berlin  . Dân số 

In [13]:
features = convert_examples_to_features(examples, tokenizer,
            cls_token_at_end=bool(MODEL_TYPE in ['xlnet']),            # xlnet has a cls token at the end
            cls_token=tokenizer.cls_token,
            sep_token=tokenizer.sep_token,
            cls_token_segment_id=2 if MODEL_TYPE in ['xlnet'] else 1,
            pad_on_left=bool(MODEL_TYPE in ['xlnet']),                 # pad on the left for xlnet
            pad_token_segment_id=4 if MODEL_TYPE in ['xlnet'] else 0)

# Convert to Tensors and build dataset
all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
all_label_ids = torch.tensor([f.label_id for f in features], dtype=torch.long)

dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)

2019-11-22 02:02:02,252 - load_data_for_bas.py - INFO - Writing example 0 of 5423


I1122 02:02:02.252789 139969067321152 load_data_for_bas.py:96] Writing example 0 of 5423


2019-11-22 02:02:02,256 - load_data_for_bas.py - INFO - *** Example ***


I1122 02:02:02.256783 139969067321152 load_data_for_bas.py:143] *** Example ***


2019-11-22 02:02:02,258 - load_data_for_bas.py - INFO - guid: 


...

I1122 02:02:02.310176 139969067321152 load_data_for_bas.py:148] input_mask: 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1


2019-11-22 02:02:02,311 - load_data_for_bas.py - INFO - segment_ids: 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1


I1122 02:02:02.311374 139969067321152 load_data_for_bas.py:149] segment_ids: 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1


2019-11-22 02:02:02,312 - load_data_for_bas.py - INFO - label: 0 (id = 0)


I1122 02:02:02.312493 139969067321152 load_data_for_bas.py:150] label: 0 (id = 0)


In [32]:
out_dir = "runs/cnn_04/"
config = BertConfig.from_pretrained(out_dir, num_labels=2, finetuning_task="zalo")
model = BertCNN_BAS.from_pretrained(out_dir, from_tf=False, config=config)

I1122 02:12:10.178758 139969067321152 configuration_utils.py:148] loading configuration file runs/cnn_04/config.json
I1122 02:12:10.180161 139969067321152 configuration_utils.py:168] Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "finetuning_task": "zalo",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bfloat16": false,
  "vocab_size": 119547
}

I1122 02:12:10.180972 139969067321152 modeling_utils.py:334] loadi

In [33]:
distill_sampler = SequentialSampler(dataset)
distill_dataloader = DataLoader(dataset, sampler=distill_sampler, batch_size=BATCH_SIZE)

In [34]:
model.to(DEVICE)
model.eval()
i = 2

out_sent = []
out_logit_0 = []
out_logit_1 = []

out_label = []
for batch in tqdm(distill_dataloader):
    batch = tuple(t.to(DEVICE) for t in batch)
    with torch.no_grad():
        inputs = {'input_ids':      batch[0],
                      'attention_mask': batch[1],
                      'token_type_ids': batch[2] if MODEL_TYPE in ['bert', 'xlnet'] else None,  # XLM don't use segment_ids
                      'labels':         batch[3]}
        try:
            model.batch_size = len(batch[0])
            model.lstm_hidden_1 = model.init_hidden()
            model.lstm_hidden_2 = model.init_hidden()
        except:
            pass
        outputs = model(**inputs)
        _, logits = outputs[:2]
        out_label.extend([0 if x[0] > x[1] else 1 for x in logits])
        out_logit_0.append([x[0] for x in logits])
        out_logit_1.append([x[1] for x in logits])

100%|██████████| 678/678 [01:37<00:00,  7.24it/s]


In [35]:
for k, l in zip(output_ids, out_label):
    k[out_dir] = l

In [37]:
df = pd.DataFrame(output_ids)
df.loc[:,'total'] = df.sum(numeric_only=True, axis=1)
df.to_csv("data/submit_cnn.csv", index=False)